In [435]:
import pandas as pd

df = pd.read_csv('BTC-USD_17.09.2014-14.03.2024.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%y')

In [436]:
from datetime import datetime

def prompt_for_date(prompt_message, comparison_date=None, must_be_after=False, max_date=None):
    while True:
        date_input = input(prompt_message)
        try:
            date = datetime.strptime(date_input, "%d-%m-%Y")
            if comparison_date is not None:
                if must_be_after and date <= comparison_date:
                    print("The date must be after %s." % comparison_date.strftime("%d-%m-%Y"))
                    continue
                elif not must_be_after and date < comparison_date:
                    print("The date must be after or equal to %s." % comparison_date.strftime("%d-%m-%Y"))
                    continue
            if max_date is not None and date > max_date:
                print("The date must not be after %s." % max_date.strftime("%d-%m-%Y"))
                continue
            return date
        except ValueError:
            print("Invalid date format. Please use the format dd-mm-yyyy.")

min_allowed_start_date = datetime.strptime("06-04-2015", "%d-%m-%Y")
max_allowed_end_date = datetime.strptime("14-03-2024", "%d-%m-%Y")

start_date = prompt_for_date("Enter the start date (dd-mm-yyyy). Starting date cannot be before 06-04-2015: ", min_allowed_start_date, must_be_after=False)
end_date = prompt_for_date("Enter the end date (dd-mm-yyyy). End date cannot be after 14-03-2024 and must be after the start date: ", start_date, must_be_after=True, max_date=max_allowed_end_date)

def prompt_for_number(prompt_message, minimum=None, condition=lambda x: True):
    while True:
        try:
            number = float(input(prompt_message))
            if minimum is not None and number <= minimum:
                print("The value must be greater than %s." % minimum)
                continue
            if not condition(number):
                print("The input does not meet the required condition.")
                continue
            return number
        except ValueError:
            print("Please enter a valid number.")

initial_balance = prompt_for_number("Enter the initial balance of the trading bot wallet: ", minimum=0)
profit_percent = prompt_for_number("Enter the % profit (value must be >= 1): ", minimum=1.00) 

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

ema_start_date = start_date - pd.Timedelta(days=200)
ema_end_date = end_date - pd.Timedelta(days=200)
initial_ema_start_date = start_date - pd.Timedelta(days=200)

start_date_df_ascending_6month = pd.to_datetime('2020-10-01')
end_date_df_ascending_6month  = pd.to_datetime('2021-04-01')
start_date_df_descending_6month = pd.to_datetime('2021-10-01')
end_date_df_descending_6month  = pd.to_datetime('2022-09-01')
start_date_df_stagnation_6month = pd.to_datetime('2015-11-01')
end_date_df_stagnation_6month  = pd.to_datetime('2016-04-01')
start_date_df_combined_1year = pd.to_datetime('2021-10-01')
end_date_df_combined_1year  = pd.to_datetime('2022-10-01')


In [437]:
def prepare_and_export(df, file_name, initial_balance, initial_profit):
    df['Position'] = 0
    df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume', 'Position': 'position'}, inplace=True)
    df.insert(1, 'time', '00:00')
    df['date'] = df['date'].dt.strftime('%m/%d/%Y')
    df['date'] = pd.to_datetime(df['date'])
    df.drop(columns=['Adj Close'], inplace=True)
    df['initial_balance'] = initial_balance
    df['profit'] = initial_profit
    df.to_csv(file_name, index=False)

def prepare_ema_data(df):
    df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume', 'Prev_Open': 'prevOpen', 'Prev_Close': 'prevClose', 'Signal': 'signal', 'Position': 'position'}, inplace=True)
    df.insert(1, 'time', '00:00')
    df['date'] = df['date'].dt.strftime('%m/%d/%Y')
    df.drop(columns=['Adj Close'], inplace=True)
    
ema_filtered_df = df[(df['Date'] >= initial_ema_start_date) & (df['Date'] <= end_date)]
ema_filtered_df_ascending_6month = df[(df['Date'] >= start_date_df_ascending_6month - pd.Timedelta(days=200)) & (df['Date'] <= end_date_df_ascending_6month)]
ema_filtered_df_descending_6month = df[(df['Date'] >= start_date_df_descending_6month - pd.Timedelta(days=200)) & (df['Date'] <= end_date_df_descending_6month)]
ema_filtered_df_stagnation_6month = df[(df['Date'] >= start_date_df_stagnation_6month - pd.Timedelta(days=200)) & (df['Date'] <= end_date_df_stagnation_6month)]
ema_filtered_df_combined_1year = df[(df['Date'] >= start_date_df_combined_1year - pd.Timedelta(days=200)) & (df['Date'] <= end_date_df_combined_1year)]
    
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()
filtered_df_ascending_6month = df[(df['Date'] >= start_date_df_ascending_6month) & (df['Date'] <= end_date_df_ascending_6month)].copy()
filtered_df_descending_6month = df[(df['Date'] >= start_date_df_descending_6month) & (df['Date'] <= end_date_df_descending_6month)].copy()
filtered_df_stagnation_6month = df[(df['Date'] >= start_date_df_stagnation_6month) & (df['Date'] <= end_date_df_stagnation_6month)].copy()
filtered_df_combined_1year = df[(df['Date'] >= start_date_df_combined_1year) & (df['Date'] <= end_date_df_combined_1year)].copy()

prepare_and_export(filtered_df, 'btcusd_tb4_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_ascending_6month, 'btcusd_tb4_asc_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_descending_6month, 'btcusd_tb4_dsc_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_stagnation_6month, 'btcusd_tb4_stg_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_combined_1year, 'btcusd_tb4_com_data.csv', initial_balance, profit_percent)

prepare_ema_data(ema_filtered_df)
prepare_ema_data(ema_filtered_df_ascending_6month)
prepare_ema_data(ema_filtered_df_descending_6month)
prepare_ema_data(ema_filtered_df_stagnation_6month)
prepare_ema_data(ema_filtered_df_combined_1year)

C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\224724209.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low','Close': 'close', 'Volume': 'volume', 'Prev_Open': 'prevOpen', 'Prev_Close': 'prevClose', 'Signal': 'signal', 'Position': 'position'}, inplace=True)
C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\224724209.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].dt.strftime('%m/%d/%Y')
C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\224724209.py:16: Se

In [438]:
def calculateEmaValues(outputDataSet, inputDataSet):
    ema_lengths = [1, 20, 25, 30, 35, 40, 45, 50, 55, 200]
    ema_values = {}
    for length in ema_lengths:
        ema = inputDataSet['close'].ewm(span=length, adjust=False).mean()
        ema_values[f'ema-{length}'] = ema
        outputDataSet[f'ema-{length}'] = ema

    return ema_values

In [439]:
ema_values_filtered = calculateEmaValues(filtered_df, ema_filtered_df)
ema_values_filtered_asc = calculateEmaValues(filtered_df_ascending_6month, ema_filtered_df_ascending_6month)
ema_values_filtered_dsc = calculateEmaValues(filtered_df_descending_6month, ema_filtered_df_descending_6month)
ema_values_filtered_stg = calculateEmaValues(filtered_df_stagnation_6month, ema_filtered_df_stagnation_6month)
ema_values_filtered_com = calculateEmaValues(filtered_df_combined_1year, ema_filtered_df_combined_1year)

In [440]:
def calculate_obv(dataset, obv_period=3):
    dataset['obv_change'] = 0
    dataset.loc[dataset['close'] > dataset['close'].shift(1), 'obv_change'] = dataset['volume']
    dataset.loc[dataset['close'] < dataset['close'].shift(1), 'obv_change'] = -dataset['volume']
    dataset['obv'] = dataset['obv_change'].cumsum()

    dataset.drop('obv_change', axis=1, inplace=True)
    
    dataset['obv_rising'] = dataset['obv'] > dataset['obv'].shift(1)
    dataset['obv_falling'] = dataset['obv'] < dataset['obv'].shift(1)
    for i in range(2, obv_period + 1):
        dataset['obv_rising'] = dataset['obv_rising'] & (dataset['obv'] > dataset['obv'].shift(i))
    for i in range(2, obv_period + 1):
        dataset['obv_falling'] = dataset['obv_falling'] & (dataset['obv'] < dataset['obv'].shift(i))
    
    dataset.fillna(method='bfill', inplace=True) 

In [441]:
calculate_obv(filtered_df, 3)
calculate_obv(filtered_df_ascending_6month, 3)
calculate_obv(filtered_df_descending_6month, 3)
calculate_obv(filtered_df_stagnation_6month, 3)
calculate_obv(filtered_df_combined_1year, 3)

C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\3380492450.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset.fillna(method='bfill', inplace=True)
C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\3380492450.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset.fillna(method='bfill', inplace=True)
C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\3380492450.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset.fillna(method='bfill', inplace=True)
C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\3380492450.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset.fillna(method='bfill', inplace=True)


In [442]:
def calculate_rsi(dataSet, period=3):
    delta = dataSet['close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [443]:
filtered_df['rsi'] = calculate_rsi(filtered_df)
filtered_df_ascending_6month['rsi'] = calculate_rsi(filtered_df_ascending_6month)
filtered_df_descending_6month['rsi'] = calculate_rsi(filtered_df_descending_6month)
filtered_df_stagnation_6month['rsi'] = calculate_rsi(filtered_df_stagnation_6month)
filtered_df_combined_1year['rsi'] = calculate_rsi(filtered_df_combined_1year)

In [444]:
def checkMarketTrend(dataSet):
    dataSet['trend'] = dataSet[['ema-20', 'ema-25', 'ema-30', 'ema-35', 'ema-40', 'ema-45', 'ema-50', 'ema-55']].apply(lambda x: all(x[i] > x[i+1] for i in range(len(x)-1)), axis=1).astype(int)

In [445]:
checkMarketTrend(filtered_df)
checkMarketTrend(filtered_df_ascending_6month)
checkMarketTrend(filtered_df_descending_6month)
checkMarketTrend(filtered_df_stagnation_6month)
checkMarketTrend(filtered_df_combined_1year)

C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\1264709472.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dataSet['trend'] = dataSet[['ema-20', 'ema-25', 'ema-30', 'ema-35', 'ema-40', 'ema-45', 'ema-50', 'ema-55']].apply(lambda x: all(x[i] > x[i+1] for i in range(len(x)-1)), axis=1).astype(int)
C:\Users\Pako\AppData\Local\Temp\ipykernel_11792\1264709472.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dataSet['trend'] = dataSet[['ema-20', 'ema-25', 'ema-30', 'ema-35', 'ema-40', 'ema-45', 'ema-50', 'ema-55']].apply(lambda x: all(x[i] > x[i+1] for i in range(len(x)-1)), axis=1).astype(int)
C:\Users\Pako\AppDat

In [446]:
#BUY SIGNAL
    #close price above 200 EMA
    #Ribbon also above 200 EMA. Price must pull back to ribbon without closing below 200 EMA in the last [x] values
    #RSI must be oversold
    #obv must be rising which indicates bullish pattern
    
#SHORT SIGNAL IS OPPOSITE

def generate_signal(row, pullback_period=5):
    ema_lengths = [20, 25, 30, 35, 40, 45, 50, 55]
    
    buy_condition_1 = row['close'] > row['ema-200']
    buy_condition_2 = all(row[f'ema-{i}'] > row['ema-200'] for i in ema_lengths)
    buy_condition_3 = all(row['rsi'] >= 60 for _ in range(pullback_period))
    buy_condition_4 = row['obv_rising']

    short_condition_1 = row['close'] < row['ema-200']
    short_condition_2 = all(row[f'ema-{i}'] < row['ema-200'] for i in ema_lengths)
    short_condition_3 = all(row['rsi'] <=40 for _ in range(pullback_period))
    short_condition_4 = row['obv_falling']

    ribbon_pullback_condition_long = all(row[f'ema-{i}'] > row['ema-200'] for i in ema_lengths)

    ribbon_pullback_condition_short = all(row[f'ema-{i}'] < row['ema-200'] for i in ema_lengths)
    
    if buy_condition_1 and buy_condition_2 and buy_condition_3 and ribbon_pullback_condition_long and buy_condition_4:
        return 1

    elif short_condition_1 and short_condition_2 and short_condition_3 and ribbon_pullback_condition_short and short_condition_4:
        return 0

    else:
        return 2


In [447]:
filtered_df['signal'] = filtered_df.apply(generate_signal, axis=1)
filtered_df_ascending_6month['signal'] = filtered_df_ascending_6month.apply(generate_signal, axis=1)
filtered_df_descending_6month['signal'] = filtered_df_descending_6month.apply(generate_signal, axis=1)
filtered_df_stagnation_6month['signal'] = filtered_df_stagnation_6month.apply(generate_signal, axis=1)
filtered_df_combined_1year['signal'] = filtered_df_combined_1year.apply(generate_signal, axis=1)

In [448]:
import time
from datetime import datetime

def trading_bot_4(balance, data, fee, profit_percent, risk_reward_ratio=3):
    trades = []
    holding = 0
    last_buy_balance = balance
    mode = ''
    holding_count = 0
    stop_loss_triggered = False

    for index, row in data.iterrows():
        date = row['date']
        unix_format_date = int(time.mktime(date.timetuple()))
        open_price = row['open']
        high_price = row['high']
        low_price = row['low']
        close_price = row['close']

        # OPEN POSITION
        if holding == 0 and (row['signal'] == 1 or row['signal'] == 0) and not stop_loss_triggered:
            buy_price = open_price
            fee_amount = (fee / 100) * balance
            real_balance = last_buy_balance - fee_amount
            last_buy_balance = real_balance
            holding = real_balance / buy_price
            mode = 'LONG' if row['signal'] == 1 else 'SHORT'

            # SET STOP LOSS
            stop_loss_level = buy_price - (buy_price * profit_percent / (100 * risk_reward_ratio)) if mode == 'LONG' else buy_price + (buy_price * profit_percent / (100 * risk_reward_ratio))

            trades.append([unix_format_date, date, mode, buy_price, open_price, high_price, low_price, close_price, holding, real_balance])
        elif holding > 0:
            current_value = holding * close_price
            profit_amount = 0
            if mode == 'SHORT':
                profit_amount = holding * (buy_price - close_price)
            else:
                profit_amount = holding * (close_price - buy_price)
            profit = profit_amount / last_buy_balance * 100
            trade_closed = False
            
            if mode == 'LONG' and low_price <= stop_loss_level:
                close_price = stop_loss_level
                stop_loss_triggered = True
            elif mode == 'SHORT' and high_price >= stop_loss_level:
                close_price = stop_loss_level
                stop_loss_triggered = True
            elif profit >= profit_percent:
                trade_closed = True

            if trade_closed or stop_loss_triggered:
                if mode == 'LONG':
                    real_balance = current_value - (fee / 100) * current_value
                else:
                    profit_amount = holding * (buy_price - close_price)
                    real_balance = last_buy_balance + profit_amount - (fee / 100) * (last_buy_balance + profit_amount)
                
                trades.append([unix_format_date, date, 'CLOSE' if mode == 'LONG' else 'COVER', close_price, open_price, high_price, low_price, close_price, holding, real_balance])
                holding = 0
                mode = ''
                last_buy_balance = real_balance
                holding_count = 0
                trade_closed = False
                stop_loss_triggered = False
            else:
                holding_count += 1

    final_balance = trades[-1][-1] if trades else balance
    return final_balance, trades


In [449]:
print(f"Initial Balance: ${initial_balance}")

final_balance_tb1, trades = trading_bot_4(initial_balance, filtered_df, 1, profit_percent)
print(f"Final Balance Custom Data: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_emarsiobv_tb4_trades.csv', index=False)

final_balance_tb1, trades_asc = trading_bot_4(initial_balance, filtered_df_ascending_6month, 1, profit_percent)
print(f"Final Balance ASC_6M: ${final_balance_tb1}")
trades_df = pd.DataFrame(trades_asc, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_emarsiobv_tb4_asc_trades.csv', index=False)

final_balance_tb2, trades_dsc = trading_bot_4(initial_balance, filtered_df_descending_6month, 1, profit_percent)
print(f"Final Balance DSC_6M: ${final_balance_tb2}")
trades_df = pd.DataFrame(trades_dsc, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_emarsiobv_tb4_dsc_trades.csv', index=False)

final_balance_tb3, trades_stg = trading_bot_4(initial_balance, filtered_df_stagnation_6month, 1, profit_percent)
print(f"Final Balance STG_6M: ${final_balance_tb3}")
trades_df = pd.DataFrame(trades_stg, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_emarsiobv_tb4_stg_trades.csv', index=False)

final_balance_tb4, trades_com = trading_bot_4(initial_balance, filtered_df_combined_1year, 1, profit_percent)
print(f"Final Balance COM_1Y: ${final_balance_tb4}")
trades_df = pd.DataFrame(trades_com, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_emarsiobv_tb4_com_trades.csv', index=False)

Initial Balance: $100.0
Final Balance Custom Data: $4305017.654990373
Final Balance ASC_6M: $492.5592968343822
Final Balance DSC_6M: $266.60668582998306
Final Balance STG_6M: $125.13831957260356
Final Balance COM_1Y: $278.11816926874604
